In [ ]:
!git clone https://github.com/Swastik166/TANS_trial.git
    

In [1]:
import sys
sys.path.insert(0, './TANS_trial')
!pip install config

In [ ]:
%%bash
cd TANS_trial
python3 main.py --gpu 0 --mode train --batch-size 140 --n-epochs 10000 --base-path path/for/storing/outcomes/ --data-path path/to/processed/dataset/is/stored/ --model-zoo path/to/model_zoo.pt --seed 777 

In [1]:
import random
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
from PIL import Image

batch_size = 32
data_path = '/kaggle/input/ie-643-tans-train-sub-dataset'


'''def get_loader(mode='train'):
    dataset = ZooDatasets(mode=mode)
    root_path = dataset.load_data()
    loader = ImageFolder(root=root_path,transform=transforms.Compose([
                                                  transforms.Resize((224, 224)),
                                                  transforms.ToTensor(),
                                              ]),
                                              )
    loader = DataLoader(dataset=loader,
                        batch_size=batch_size,
                        shuffle=(mode == 'train'),
                        num_workers=4)
    return dataset, loader'''


class ZooDatasets(Dataset):
    
    def __init__(self, mode='train', transform=None):
        #self.args = args
        self.mode = mode
        self.transform = transform
        self.data_path = data_path
        '''self.dataset_list = [
                'bottles',
                'cassava_leaf_disease',
                'casting_products',
                'corals',
                'ct_images',
                'four_shapes',
                'fruits',
                'lego_bricks',
                'natural_images',
                'store_items',
        ]'''
        self.dataset_list = ['corals','lego_bricks']
        
        self.curr_dataset = self.dataset_list[0]
        self.load_data()


              
        
    def get_loader(self, mode='train'):
        root_path = self.load_data()
        loader = ImageFolder(root=root_path, transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]))

        loader = DataLoader(
            dataset=loader,
            batch_size=batch_size,
            shuffle=(mode == 'train'),
            num_workers=4
        )
        
        return loader
        
    def load_data(self):
        if self.mode == 'train':
            self.data_folder = os.path.join(self.data_path, f'{self.curr_dataset}/tr')
        elif self.mode == 'validation':
            self.data_folder = os.path.join(self.data_path, f'{self.curr_dataset}/va')
        elif self.mode == 'test':
            self.data_folder = os.path.join(self.data_path, f'{self.curr_dataset}/te')
            
            
        self.classes = sorted([d.name for d in os.scandir(self.data_folder) if d.is_dir()])
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.samples = []
        
        for target_class in self.classes:
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(self.data_folder, target_class)
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    # Modify here to convert label to integer
                    label = self.class_to_idx[target_class]
                    self.samples.append((path, label))
        
        
        return self.data_folder
        
     
        
    def get_dataset_list(self):
        return self.dataset_list
    
    
    def curr(self):
        return self.curr_dataset
    
    def set_mode(self, mode):
        self.mode = mode
        
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = Image.open(path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
            

        return img, target
    
    
    def set_dataset(self, dataset):
        self.curr_dataset = dataset
        self.load_data()
            
    def get_nclss(self):
        return (len(self.classes))
    
    def get_clss(self):
        return (self.classes)
        

In [8]:
tr_dataset = ZooDatasets(mode='train')
te_dataset = ZooDatasets(mode='test')
val_dataset = ZooDatasets(mode='validation')

'''intr_loader = intr_dataset.get_loader(mode='train')
inte_loader = inte_dataset.get_loader(mode='test')
inval_loader = inval_dataset.get_loader(mode='validation')'''

print('outer', tr_dataset.curr_dataset)

for query_dataset in tr_dataset.get_dataset_list():
    print('quesry_dataset',query_dataset)
    tr_dataset.set_dataset(query_dataset)
    te_dataset.set_dataset(query_dataset)
    val_dataset.set_dataset(query_dataset)
    
    tr_loader = tr_dataset.get_loader(mode='train')
    te_loader = te_dataset.get_loader(mode='test')
    val_loader = val_dataset.get_loader(mode='validation')
    
    
    print(tr_dataset.curr_dataset)
    x, y = next(iter(tr_loader))
    print('y for tr', x.shape)
    x, y = next(iter(te_loader))
    print('y for te', y)
    
    clss = tr_dataset.get_clss()
    nclss = tr_dataset.get_nclss()
    
    print('nclass', nclss )
    print('clss', clss )
    

    print('tr_loader', len(tr_loader)*batch_size)
    print('te_loader', len(te_loader)*batch_size)
    print('val_loader', len(val_loader)*batch_size)

outer corals
quesry_dataset corals
corals
y for tr torch.Size([32, 3, 224, 224])
y for te tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3,
        3, 3, 3, 4, 4, 5, 5, 5])
nclass 14
clss ['ACER', 'APAL', 'CNAT', 'DANT', 'DSTR', 'GORG', 'MALC', 'MCAV', 'MMEA', 'MONT', 'PALY', 'SPO', 'SSID', 'TUNI']
tr_loader 512
te_loader 96
val_loader 64


In [2]:
import torch
import numpy as np
import random
import torchvision.models as models
from tqdm import tqdm
import time
!pip install gdown

"""add utils from misc and use the finctions available in for better error handling"""

noise_path = '/kaggle/working/noise.pt'
learn = 1e-2
patience = 5
n_eps_finetuning = 5

class ModelZoo:
    def __init__(self):
        self.seed = 777
        self.models = {}
        self.train_instances = {}
        self.trainpt_dict ={}
        
        self.noise = torch.load(noise_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        torch.cuda.manual_seed(self.seed)
        torch.manual_seed(self.seed)
        np.random.seed(self.seed)
        random.seed(self.seed)


        
        
    def init_loaders(self):
        # Get loaders for train, test, and validation data
        print('\n=====> LOADING dataset and loaders for train, test and validation <=====\n')
       
        
        self.tr_dataset = ZooDatasets(mode='train')
        self.te_dataset = ZooDatasets(mode='test')
        self.val_dataset = ZooDatasets(mode='validation')
        
        
        
        
        
        
    def create_zoo(self):
        zoo = { 'dataset': [],
                 'topol': [],
                    'f_emb': [],
                   'acc': [],
                   'n_params':[]
            
        }
        
        
        self.init_loaders()

            
            
        for query_dataset in self.tr_dataset.get_dataset_list():
            
            
            
            print('quesry_dataset',query_dataset)
            
            self.tr_dataset.set_dataset(query_dataset)
            self.te_dataset.set_dataset(query_dataset)
            self.val_dataset.set_dataset(query_dataset)

            tr_loader = self.tr_dataset.get_loader(mode='train')
            te_loader = self.te_dataset.get_loader(mode='test')
            val_loader = self.val_dataset.get_loader(mode='validation')
            nclass = self.tr_dataset.get_nclss()
            print(f"\nDataset : {query_dataset}, nclss:{nclass}\n")
            
            
            


            for i in range(3):
                # Get neural network model and topology information
                print(f'\n=====> Generating {i+1}th network  <=====\n')
                topol, net = self.get_net(nclass)

                # Training the model and obtaining accuracy
                lss = torch.nn.CrossEntropyLoss()
                optim = torch.optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=4e-5)
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim , float(n_eps_finetuning))
                
                acc = self.train( net, optim, scheduler, lss, query_dataset, tr_loader, val_loader, nclass, topol)  # Assuming train method returns accuracy

                # Calculating number of parameters
                n_params = self.n_param(net)
                f_emb = self.f_emb(net)

                
                del net
                del optim
                del lss
                
                zoo['dataset'].append(query_dataset)
                zoo['topol'].append(topol)
                zoo['acc'].append(acc)
                zoo['f_emb'].append(f_emb)
                zoo['n_params'].append(n_params)
                
                print(zoo)
                
            x_query_train, x_query_test = self.get_query(tr_loader, te_loader)
            print(f'x_query:{len(x_query_train)}, x_query-test :{len(x_query_test)}')
            clss = self.tr_dataset.get_clss()
            self.save_trainpt(query_dataset, clss, nclass, x_query_test, x_query_train)
                
        self.save_zoo(zoo)
        
                
                


    def get_net(self, nclss):
        print("\n=====>Generating sub_net<=====\n")
        super_net_name = "ofa_supernet_mbv3_w12"

        # Load the super network from the 'mit-han-lab/once-for-all' repository
        super_net = torch.hub.load('mit-han-lab/once-for-all', super_net_name, pretrained=True).eval()

        # Sample an active subnet configuration from the super network
        sampled_config = super_net.sample_active_subnet()

        # Extract topology information from the sampled configuration
        pre_topol = list(sampled_config.values())
        topol = []

        for i in pre_topol:
            for j in i:
                topol.append(j)

        # Split the topology into kernel sizes, expansion ratios, and depths
        ks = topol[:20] 
        e = topol[20:40]
        d = topol[40:]

        # Set the active subnet in the super network using the sampled topology
        super_net.set_active_subnet(ks=ks, e=e, d=d)
        active_subnet = super_net.get_active_subnet(preserve_weight=True)
        active_subnet.classifier = torch.nn.Linear(1536, nclss)
        active_subnet = active_subnet.to(self.device)

        return topol, active_subnet

    
    
    
        '''def f_emb(self, net):
        print("=====>Generating f_emb<=====")
        model = net
        model.to(self.device)
        modules = list(model.children())
        last_valid_module = None

        # Find the last valid module with a defined forward function
        for module in reversed(modules):
            if hasattr(module, 'forward'):
                last_valid_module = module
                break

        if last_valid_module is None:
            raise NotImplementedError("No valid module with a 'forward' function found.")

        # Remove the last layer (classifier)
        modified_model = torch.nn.Sequential(*modules[:-1]) if len(modules) > 1 else last_valid_module

        modified_model.eval()
        with torch.no_grad():
            # Forward pass through the modified model
            noise = self.noise.to(self.device)
            f_emb = modified_model(noise)
        print(f_emb)
        return f_emb'''
        
        

        

    def f_emb(self, net):
        print("\n=====>Generating f_emb<=====\n")
        model = net
        model.to(self.device)
        layer = model.feature_mix_layer
        
        def copy_embeddings(module, input, output):
            # Assuming the output shape is (batch_size, channels, height, width)
            embeddings = output.squeeze()  # Remove dimensions of size 1
            outputs.append(embeddings.cpu().detach().numpy())


        outputs = []
        _ = layer.register_forward_hook(copy_embeddings)

        model.eval()
        noise = self.noise.to(self.device)
        _ = model(noise)
        f_emb = outputs[-1]
        f_emb = torch.tensor(f_emb)
        del outputs
        print(f_emb.shape)
        return f_emb
    
    
    

    def train(self, model, optim, scheduler, lss, dataset, train_loader, val_loader, nclss, topol):
        print("\n=====> Training started <=====\n")
        #self.model = model
        print(f'Starting Training for:{dataset} with model topology:{topol}')
        lr = learn
        counter = 0
        best_val_loss = 10000
        val_acc = list()
        
        
        for ep in range(n_eps_finetuning):
            curr_ep = ep
            ep_loss_tr = 0.0
            ep_loss_val = 0.0
            ep_tr_time = 0
            st = time.time()
            
            
            model.train()
            for b_id, batch in tqdm(enumerate(train_loader)):
                optim.zero_grad()
                
                x,y = batch
                output = model(x.to(self.device))
                loss = lss(output, y.to(self.device))
                loss.backward()
                optim.step()
                scheduler.step()
                        
                tr_loss = loss.item()
                        
                ep_loss_tr += tr_loss * x.size(0)
                
            ep_loss_tr = ep_loss_tr/len(train_loader)

                        
            model.eval()
            total_val = 0
            correct_val = 0
                        
                        
            for v_id, (x,y) in tqdm(enumerate(val_loader)):
                outputs = model(x.to(self.device))
                loss_v = lss(outputs, y.to(self.device))
                       
                val_loss = loss_v.item()
                
                ep_loss_val += val_loss * x.size(0)
                        
                pred = torch.argmax(outputs, dim = 1)
                total_val += y.size(0)
                correct_val += (pred == y.to(self.device)).sum().item() 
                        
            acc = (100*correct_val)/total_val
            val_acc.append(acc)
            ep_loss_val = ep_loss_val/len(val_loader)
            dura = time.time() -st 
            
            print('\nEpoch: {}/{}, Train Loss: {:.8f} , Val Loss: {:.8f}, Val Accuracy: {:.8f}, Time: {:.8f}'.format(ep + 1, n_eps_finetuning, ep_loss_tr, ep_loss_val, acc, dura))
            
            
    
            if ep_loss_val < best_val_loss:
                best_val_loss = ep_loss_tr
                counter = 0
                
            else:
                counter += 1
                
            if counter >= patience:
                print(f"Early stopping on, {ep}th, epoch")
                break
               
            #print(f'ep : {ep}, loss:{ep_loss_tr}, val_loss:{ep_loss_val}, acc:{acc}, time:{dura}')
                
        return val_acc[-1]
            
            
                        
                        
                        
                        
    
    def n_param(self, model):
        # Calculate the number of parameters in the model
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    

    def get_query(self, train_loader, test_loader):
        print("\n=====> GENERATING QUERIES <=====\n")
        model = models.resnet18(pretrained=True)
        layer = model._modules.get('avgpool')

        def copy_embeddings(m, i, o):
            """Copy embeddings from the penultimate layer."""
            o = o[:, :, 0, 0].detach().numpy().tolist()
            outputs.append(o)

        outputs = []
        # attach hook to the penultimate layer
        _ = layer.register_forward_hook(copy_embeddings)

        model.eval()
        X, y = next(iter(train_loader))
        _ = model(X)

        list_embeddings = [item for sublist in outputs for item in sublist]

        query_train = []
        for a in list_embeddings:
            a = torch.tensor(a)
            query_train.append(a)

        print(f'query_shape_train: {len(query_train)}, each tensor:{query_train[0].shape}')

        outputs = []
        list_embeddings = []

        model.eval()

        X, y = next(iter(test_loader))
        _ = model(X)

        list_embeddings = [item for sublist in outputs for item in sublist]

        query_test = []
        for a in list_embeddings:
            a = torch.tensor(a)
            query_test.append(a)

        print(f'query_shape_test: {len(query_test)}, each tensor:{query_test[0].shape}')
        del outputs
        del list_embeddings
        return query_train, query_test
    
    '''def get_query(self, train_loader, test_loader):
        print("\n=====>GENERATING QUERIES<=====\n")
        model = models.resnet18(pretrained=True)
        module = list(model.children())[:-1]
        model = torch.nn.Sequential(*module)
        model.eval()
        
        with torch.no_grad():
            for X, y in train_loader:
                query_train = model(X)
                query_train = torch.squeeze(query_train)
                break  # Exit loop after processing the first batch

        print(f'query_train_shape: {query_train.shape}')

        with torch.no_grad():
            for X, y in test_loader:
                query_test = model(X)
                query_test = torch.squeeze(query_test)
                break  # Exit loop after processing the first batch

        print(f'query_test_shape: {query_test.shape}')

        return query_train, query_test'''
    
    
    

    def save_zoo(self, dict):
        # Save model zoo to model_zoo.pt
        print(f'\n=====>SAVING {dict} in model_zoo.pt <=====\n')

        torch.save(dict, 'model_zoo.pt')

        
        
    def save_trainpt(self, dataset, clss, nclss, x_test, x_train) :
        # Save training instances to m_train.pt
        print(f'\n=====>SAVING {dataset} in m_train.pt <=====\n')
        temp = {}
        temp['task'] = dataset
        temp['clss'] = clss
        temp['nclss'] = nclss
        temp['x_query_test'] = x_test
        temp['x_query_train'] = x_train
        
        self.trainpt_dict[dataset] = temp
        

        torch.save(self.trainpt_dict, 'meta_train.pt')


In [4]:
zoo = ModelZoo()
zoo.create_zoo()


=====> LOADING dataset and loaders for train, test and validation <=====

quesry_dataset corals

Dataset : corals, nclss:14


=====> Generating 1th network  <=====


=====>Generating sub_net<=====



/opt/conda/lib/python3.10/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/mit-han-lab/once-for-all/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://raw.githubusercontent.com/han-cai/files/master/ofa/ofa_nets/ofa_mbv3_d234_e346_k357_w1.2" to .torch/ofa_nets/ofa_mbv3_d234_e346_k357_w1.2



=====> Training started <=====

Starting Training for:corals with model topology:[3, 5, 5, 5, 7, 5, 5, 7, 3, 5, 7, 7, 7, 5, 7, 5, 3, 3, 7, 3, 4, 3, 6, 3, 6, 6, 4, 4, 6, 4, 3, 6, 6, 3, 4, 6, 3, 4, 6, 3, 3, 4, 2, 4, 4]


16it [00:08,  1.83it/s]
2it [00:00,  5.04it/s]


Epoch: 1/5, Train Loss: 73.23685668 , Val Loss: 69.62717223, Val Accuracy: 51.61290323, Time: 9.36200190


...


Epoch: 3/5, Train Loss: 2.39741317 , Val Loss: 1.48009005, Val Accuracy: 98.90282132, Time: 24.27371645



160it [00:22,  6.97it/s]
20it [00:01, 16.54it/s]


Epoch: 4/5, Train Loss: 0.98825601 , Val Loss: 0.70580279, Val Accuracy: 99.37304075, Time: 24.44176388



160it [00:22,  6.99it/s]
20it [00:01, 17.21it/s]



Epoch: 5/5, Train Loss: 0.75378004 , Val Loss: 0.99943149, Val Accuracy: 98.90282132, Time: 24.35710669

=====>Generating f_emb<=====

torch.Size([1536])
{'dataset': ['corals', 'corals', 'corals', 'lego_bricks', 'lego_bricks', 'lego_bricks'], 'topol': [[3, 5, 5, 5, 7, 5, 5, 7, 3, 5, 7, 7, 7, 5, 7, 5, 3, 3, 7, 3, 4, 3, 6, 3, 6, 6, 4, 4, 6, 4, 3, 6, 6, 3, 4, 6, 3, 4, 6, 3, 3, 4, 2, 4, 4], [3, 7, 7, 7, 5, 7, 5, 3, 7, 5, 3, 3, 5, 5, 3, 7, 7, 5, 3, 5, 3, 4, 6, 3, 6, 4, 6, 4, 4, 3, 3, 3, 3, 6, 6, 3, 3, 4, 6, 4, 4, 3, 2, 2, 4], [3, 7, 5, 5, 3, 7, 7, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 5, 7, 6, 6, 6, 3, 6, 3, 4, 4, 6, 6, 6, 4, 3, 6, 4, 6, 6, 4, 6, 4, 2, 3, 3, 3, 4], [7, 5, 3, 3, 5, 3, 5, 5, 5, 5, 3, 7, 7, 5, 7, 3, 3, 5, 3, 5, 3, 4, 6, 6, 6, 3, 6, 3, 6, 3, 6, 4, 4, 6, 6, 4, 6, 6, 6, 4, 2, 2, 2, 3, 2], [3, 5, 5, 7, 5, 5, 7, 3, 7, 5, 5, 5, 3, 5, 3, 7, 5, 7, 7, 3, 3, 6, 3, 3, 6, 3, 6, 6, 4, 4, 3, 4, 4, 3, 6, 6, 4, 3, 4, 3, 4, 3, 4, 4, 4], [3, 3, 5, 5, 7, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 7, 7, 3, 5, 5, 

In [ ]:
model_zoo = {'dataset':['dataset1', 'dataset1', 'dataset2', 'dataset2', 'dataset3', 'dataset3'],
             'topol':['model1', 'model2', 'model3', 'model4' , 'model5',  'model6'],
             'acc':['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6'],
             'f_emb':['f_emb1', 'f_emb2', 'f_emb3', 'f_emb4', 'f_emb5', 'f_emb6'],
             'n_params':['np1', 'np2', 'np3', 'np4', 'np5', 'np6']}

In [ ]:
model_zoo

In [ ]:
import torch 
torch.save(model_zoo, 'zoo.pt')

In [ ]:
zoo = torch.load('zoo.pt')
print(zoo)
print(type(zoo))

In [ ]:
zoo['dataset'].append('test')
zoo['topol'].append('test')
zoo['acc'].append('test')
zoo['f_emb'].append('test')
zoo['n_params'].append('test')
print(zoo)

In [3]:
import torch
noise = torch.randn([1, 3, 224, 224])
print(noise.shape)
torch.save(noise, 'noise.pt')

torch.Size([1, 3, 224, 224])
